In [ ]:
import os
import re
import random
#import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from itertools import count
from collections import namedtuple

%matplotlib inline

# Task 1

In [ ]:
Sample = namedtuple('Sample', ['cls', 'idx','quat','img'])
def normalize(mat):
    out = np.empty_like(mat)
    for i in range(3):
        m = np.mean(mat[:,:,i])
        v = np.var(mat[:,:,i])
        out[:,:,i] = (mat[:,:,i] - m) / v
    return out

def parse(folder, cls):
    # quats
    quats = []
    with open(os.path.join(folder,'poses.txt'),'r') as f:
        while True:
            line1 = f.readline()
            line2 = f.readline()
            if not line2: break
            quats.append(np.array([float(x) for x in line2.split()]))

    # images
    imgs = []
    p = re.compile(r'[A-Za-z]+[0-9]+\.png')
    for imname in os.listdir(folder):
        if p.match(imname):
            imgs.append(
                normalize(mpimg.imread(os.path.join(folder,imname))))
            
    return [Sample(cls, idx, quat, img) for idx, quat, img in zip(count(),quats,imgs)]

## Loading Data

In [ ]:
N_CLASSES = len(os.listdir('dataset/coarse/'))

data = []
p = re.compile(r'[A-Za-z]+[0-9]+\.png')
for root, subfolders,_ in os.walk('dataset/coarse/'):
    for i,folder in enumerate(sorted(subfolders)):
        data.append(parse(os.path.join(root,folder), i))

coarse = [l for lst in data for l in lst]

data = []
for root, subfolders,_ in os.walk('dataset/fine/'):
    for i,folder in enumerate(sorted(subfolders)):
        data.append(parse(os.path.join(root,folder), i))
        
fine = [l for lst in data for l in lst]

data = []
for root, subfolders,_ in os.walk('dataset/real/'):
    for i,folder in enumerate(sorted(subfolders)):
        data.append(parse(os.path.join(root,folder), i))

real = [l for lst in data for l in lst]

In [ ]:
train_idxs = ()
with open('dataset/real/training_split.txt') as f:
    train_idxs = set(int(x) for x in f.read().split(', '))
test_idxs = set(range(len(real))) - train_idxs

In [ ]:
Sdb    = coarse
Strain = fine + [real[i] for i in train_idxs]
Stest  = [real[i] for i in test_idxs]

In [ ]:
print(len(Sdb), len(Strain), len(Stest))

In [ ]:
def similarity(q1, q2):
    return 2 * np.arccos(min(1,np.abs(q1 @ q2)))

def batch(Sdb, Strain, n):
    def gen():
        while 1:
            # Anchor: select random sample from Strain
            anchor = random.choice(Strain)
            # Puller: select most similar from Sdb
            puller = max(Sdb, key = lambda x: similarity(x.quat,anchor.quat))
            # Pusher: same object different pose | random different object 
            pusher = random.choice([x for x in Sdb if x.cls != anchor.cls])
            yield anchor
            yield puller
            yield pusher
    g = gen()
    return [next(g) for _ in range(3*n)]

In [ ]:
#img = mpimg.imread('dataset/coarse/ape/coarse250.png')
plt.imshow(Sdb[100].img)